In [3]:
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
import random

sequences = []
generated_sequences = set()


def complement_base(base):
       if base == 'A':
              return 'T'
       elif base == 'T':
              return 'A'
       elif base == 'C':
              return 'G'
       else:
              return 'C'

def reverse_compliment(seq):
    seq_dict = {'A':'T', 'T':'A','C':'G', 'G':'C'}
    reverse_compliment = ''
    for base in seq[::-1]:
        if base == ' ': #if block to clean up the spacing in the data above, if empty space then add an empty space
            reverse_compliment += ' '
        else: #if not empty space look up the dictionary
            reverse_compliment += seq_dict[base]
    return reverse_compliment

#generate sequences
seqs = []
for c1 in "ATCG":
      for c2 in "ATCG":
            for c3 in "ATCG":
                  for c4 in "ATCG":
                        for c5 in "ATCG":
                              for c6 in "ATCG":
                                    for c7 in "ATCG":
                                          for c8 in "ATCG":
                                                seqs.append("".join([c1,c2,c3,c4,c5,c6,c7,c8]))
                                                rdict = {'A':'T', 'T':'A','C':'G', 'G':'C'}
def rc (seq):
    return "".join([
        rdict[c]
        for c in seq
    ][::-1])
forwards = [] 
for s in seqs:
    if rc(s) in forwards:
        pass
    else:
        forwards.append(s)
len(forwards) 

forwards = list(
    filter(
    lambda seq: 15 <= mt.Tm_NN(seq, Mg = 15, Na = 5) <= 17,
        forwards
    )
)
len(forwards)

#CG contents
forwards = [
    seq for seq in forwards 
    if (seq.count("C") + seq.count("G")) / len(seq) == 0.5
]
len(forwards)


2620

In [5]:
import random
from nupack import *
from tqdm import tqdm

model1 = Model(material='dna', celsius=16, sodium=0.05, magnesium=0.00)
max_unique_seq = []
max_len = 0

r_list = []

while len(r_list) < len(forwards):
    
    
    for r in range(len(forwards)):
        unique_seq = []
        potential_seq = []
        if len(unique_seq) == 0:
            unique_seq.append(forwards[r])
            unique_seq.append(reverse_complement(forwards[r]))
            r_list.append(r)

        r1_list = []
        for a in tqdm(range(1000000)):
            
            if len(r1_list) == len(forwards)-1:
                continue
            count = 0
            for b, uni_seq in enumerate(unique_seq):
                if b % 2 ==0:
                    
                    r1 = random.randint(0,len(forwards)-1)
                    if r1 not in r1_list and r1 != r:
                        potential_seq.append(forwards[r1])
                        r1_list.append(r1)

                        dG_comp_pot = pfunc((forwards[r1], reverse_complement(forwards[r1])), model=model1)
                        
                        dG_pot_f_1 = pfunc((forwards[r1], uni_seq), model = model1) #compare dG from possible pair (forward) to the sequences inside unique_seq
                        
                        dG_pot_f_2 = pfunc((forwards[r1], reverse_complement(uni_seq)), model = model1)
                        
                        dG_pot_r_1 = pfunc((reverse_complement(forwards[r1]),uni_seq),model=model1) #compare dG from possible pair (reverse) to the sequences inside unique_seq
                        
                        dG_pot_r_2 = pfunc((reverse_complement(forwards[r1]),reverse_complement(uni_seq)), model=model1)

                        dG_palindromic = pfunc((forwards[r1],forwards[r1]), model=model1)

                        ddG_pot_f_1 = dG_comp_pot[1] - dG_pot_f_1[1]
                        ddG_pot_f_2 = dG_comp_pot[1] - dG_pot_f_2[1]

                        ddG_pot_r_1 = dG_comp_pot[1] - dG_pot_r_1[1]
                        ddG_pot_r_2 = dG_comp_pot[1] - dG_pot_r_2[1]
                        
                        ddG_palindromic = dG_comp_pot[1] - dG_palindromic[1]

                        if ddG_pot_f_1 <= -3.4 and ddG_pot_f_2 <= -3.4 and ddG_pot_r_1 <= -3.4 and ddG_pot_r_2 <= -3.4 and ddG_palindromic <= -2.5:
                            # print(ddG_pot_f_1,',',ddG_pot_f_2,',',ddG_pot_r_1,',',ddG_pot_r_2)
                            count += 2
                        # print(count)
                        if count == len(unique_seq): #why unique_seq cannot more than 4
                            # print(len(unique_seq))
                            
                            count_1 = 0
                            for c, strand in enumerate(unique_seq):
                                
                                if c % 2 ==0:
                                    dG_comp_1 = pfunc((strand, reverse_complement(strand)), model=model1) #compare dG from the sequences inside unique_seq to the possible pair
                                    
                                    dG_forw_1 = pfunc((strand,potential_seq[0]), model=model1) #check forward strand inside unique with forward and reverse in the possible pair
                                    dG_forw_2 = pfunc((strand,reverse_complement(potential_seq[0])),model=model1)
                                    

                                    dG_rev_1 = pfunc((reverse_complement(strand),potential_seq[0]), model=model1) #check reverse strand inside unique with forward and reverse in the possible pair
                                    dG_rev_2 = pfunc((reverse_complement(strand),reverse_complement(forwards[r1])),model=model1)
                                    
                                    

                                    ddG_forw_1 = dG_comp_1[1] - dG_forw_1[1]
                                    ddG_forw_2 = dG_comp_1[1] - dG_forw_2[1]
                                    

                                    ddG_rev_1 = dG_comp_1[1] - dG_rev_1[1]
                                    ddG_rev_2 = dG_comp_1[1] - dG_rev_2[1]
                                    


                                    if ddG_forw_1 <= -3.4 and ddG_forw_2 <= -3.4 and ddG_rev_1 <= -3.4 and ddG_rev_2 <= -3.4:
                                        # print(ddG_forw_1,',',ddG_forw_2,',',ddG_rev_1,',',ddG_rev_2)
                                        count_1 += 2   #why always = 2
                                    
                                    if count_1 == len(unique_seq):
                                        
                                        unique_seq.append(potential_seq[0])
                                        unique_seq.append(reverse_complement(potential_seq[0]))
                                        

                            if c == len(unique_seq)-1:
                                potential_seq = []
                        
        with open('forwards_list_index', 'w') as file:
            for number in r_list:
                file.write(str(number) + ',')
            
        if len(unique_seq) > max_len or len(unique_seq) >= 48:
            max_len = len(unique_seq)
            max_unique_seq = unique_seq[:]
            with open('unique_strands.txt', 'w') as file:
                for i in range(0, len(max_unique_seq), 2):
                    print(max_unique_seq[i])
                    print(max_unique_seq[i+1])
                    file.write(max_unique_seq[i] + '\n')
                    file.write(max_unique_seq[i+1] + '\n')
            print(len(max_unique_seq))        

100%|██████████| 1000000/1000000 [00:33<00:00, 29749.01it/s] 


AAAACCCC
GGGGTTTT
TCCACTTG
CAAGTGGA
ACCTCAGA
TCTGAGGT
TCAGAAGC
GCTTCTGA
AATACGGG
CCCGTATT
TGGTACCA
TGGTACCA
TGAAGACC
GGTCTTCA
ACTTCGAG
CTCGAAGT
TGCTTACC
GGTAAGCA
AATCCTGC
GCAGGATT
CAATTCGG
CCGAATTG
ACTACTCG
CGAGTAGT
TCCATGTG
CACATGGA
AGATTGGC
GCCAATCT
28


100%|██████████| 1000000/1000000 [00:30<00:00, 32738.79it/s] 


AAAAGGGG
CCCCTTTT
ATACCGGT
ACCGGTAT
GTCAAACC
GGTTTGAC
CAAATCCG
CGGATTTG
CATAACCG
CGGTTATG
CGTTTACC
GGTAAACG
TAAGGCTG
CAGCCTTA
CTGAACAC
GTGTTCAG
AAGATCGG
CCGATCTT
TTCGAAGG
CCTTCGAA
TATGCACC
GGTGCATA
TATGTCGG
CCGACATA
ACTCAGTG
CACTGAGT
AGGAGTGA
TCACTCCT
TTCCAACC
GGTTGGAA
TCTAGCAC
GTGCTAGA
32


100%|██████████| 1000000/1000000 [00:32<00:00, 30903.01it/s] 


...

100%|██████████| 1000000/1000000 [00:37<00:00, 26581.12it/s] 


AATCCTGC
GCAGGATT
TGTATGGC
GCCATACA
CATGAAGC
GCTTCATG
AGTCCAGA
TCTGGACT
TTCCACAC
GTGTGGAA
TATTGCCC
GGGCAATA
CGTTACAC
GTGTAACG
ACGATCTC
GAGATCGT
TGTGCTAC
GTAGCACA
AGTGAACC
GGTTCACT
ACAATGGG
CCCATTGT
TCCAACTC
GAGTTGGA
TTCCTTGG
CCAAGGAA
CAAGTGTC
GACACTTG
AGCATTCC
GGAATGCT
ACGAGTAG
CTACTCGT
CAATAGCG
CGCTATTG
AGGCTTAC
GTAAGCCT
ATACGGAC
GTCCGTAT
CGATTACG
CGTAATCG
40


100%|██████████| 1000000/1000000 [00:42<00:00, 23593.81it/s] 


AACGGTAG
CTACCGTT
TTGGACAC
GTGTCCAA
ACTAGTGC
GCACTAGT
ACACTCTG
CAGAGTGT
TGACTACG
CGTAGTCA
TCATGGGA
TCCCATGA
TGACCTCA
TGAGGTCA
TCTGAACC
GGTTCAGA
CAGAATGC
GCATTCTG
AGCCAGTA
TACTGGCT
TCGAGAAG
CTTCTCGA
TTGTAGCC
GGCTACAA
ATCATCCG
CGGATGAT
CAACCTTG
CAAGGTTG
AATCAGCC
GGCTGATT
CATCTTGC
GCAAGATG
TACCCCAA
TTGGGGTA
GATCATGC
GCATGATC
TCAGTGAC
GTCACTGA
CTCTTTCG
CGAAAGAG
TTGTGGAG
CTCCACAA
42


100%|██████████| 1000000/1000000 [00:18<00:00, 54974.30it/s] 
